# Исследование надежности заемщиков.

Цель исследования - ответить на четыре вопроса:
1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

## Шаг 1. Обзор данных

Основной инструмент анализа в данном проекте — pandas. Импортирую эту библиотеку

In [1]:
import pandas as pd

Датасет представлен в формате CSV, чтобы открыть таблицу подойдет функция `read_csv()`.

Прочитаю файл `data.csv` из папки `/datasets` и сохраню его в переменной `df`:

Просмотрю первые 10 строк таблица, чтобы ознакомиться с данными

In [2]:
df = pd.read_csv('/datasets/data.csv')
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получу общую информацию о таблице.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов. В соответствии с имеющейся документацией к данным:

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

Присутствуют разные типы данных - `float`, `int` и `object`. 

Выводы:
- В названиях колонок не видно нарушений стиля;
- В стобцах total_income и days_employed отличное от других количество строк, значит присутствуют пропуски;
- В столбце education данные записаны в разных регистрах, это может повлиять на обработку значений;
- В столбце days_employed данные принимают отрицательное значение.

## Шаг 2. Предобработка данных

### Шаг 2.1 Заполнение пропусков

На предыдущем этапе обнаружено два столбца, в которых могут быть пропуски total_income и days_employed.
Проверю это, применив функции isna() и sum(), для расчета количества нулевых значений в столбцах по всей таблице.

In [4]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Действительно, в столбцах days_employed и total_income по 2174 пропущенных значения. Но вывод метод isna() осуществляет поиск только особых значений NaN и None. В выводе общей информации о таблице было указанно, что в этих столбцах содержаться значения только типа float, значит пропуски записаны как NaN. Остальные значения буду воспринимать как артефакты, их рассматрю в следующих пунктах.

Следует обратить внимае на то, что количество пропусков совпадает в обоих столбцах. Возможно пропуски находятся в одних строках. Проверю это, использовав метод loc[] для фильтрации пропусков и функцию len() для рассчета колличества строк.


In [5]:
df_missing_values = df.loc[df['days_employed'].isna() & df['total_income'].isna()]
missing_values_count = len(df_missing_values)
print('Количество пропусков:', missing_values_count)

Количество пропусков: 2174


Все пропущенные значения в столбцах days_employed и total_income находятся в одних строках. Следовательно долю пропущенных значений можно рассчитать следующим образом:

In [6]:
print('Доля пропущенных значений:', '{:.2%}'.format(df['days_employed'].isna().mean())) #f'{(missing_values_count/21525):.2%}')

Доля пропущенных значений: 10.10%


Рассмотрим строки с пропущенными значениями. Выведу на экран первые 10 строк.

In [7]:
display(df_missing_values.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Пропущенные значения могли возникнуть по нескольким причинам:
- при выгрузке данных произошла ошибка, тогда необходимо обратиться к разработчикам за помощью и исправлением ошибок, но доступа к ним сейчас у меня нет;
- человеческий фактор, данные были некорректно введены при заполнении.

Удалить пропущенные значения нельзя, так как потеряется 1/10 информации, которая содержится в других столбцах датасета. Это повлияет на результаты исследования. Пропуски необходимо заменить на типовые значения таблицы, такими могут выступать медиана или среднее.

В данных могут присутствовать артефакты или сильно выделяющиеся значения среди большинства, поэтому среднее значение будет некорректно описывать таблицу. Небходимо использовать медиану.

Заменю пропуски в столбце 'total_income' на медианное значение используя функцию fillna() и проверю, что замена прошла успешно.

In [8]:
df['total_income'] = df['total_income'].fillna(df['total_income'].median())
print('Количество пропусков в столбце total_income:', df['total_income'].isna().sum())

Количество пропусков в столбце total_income: 0


Перед заменой пропусков в столбце days_employed необходимо обработать отрицательные значения. Они могли появиться по следующим причинам:
- при рассчете трудового стажа были перепутаны даты начала и окончания работы;
- ошибки форматирования данных: даты были введены в разных форматах. Чтобы исправить эту ошибку нужны исходные данные, таких у меня нет.

Заменю отрицательные значения на их модуль, использую функцию abs() и проверю, что отрицательных значений больше нет.

In [9]:
df['days_employed'] = df['days_employed'].abs()
print('Количество отрицательных значений в столбце days_employed:', df[df['days_employed'] < 0]['days_employed'].count())

Количество отрицательных значений в столбце days_employed: 0


Заменю пропуски в столбце 'days_employed' на медианное значение, используя функцию fillna() и проверю, что пропусков больше нет.

In [10]:
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())
print('Количество пропусков в столбце days_employed:', df['days_employed'].isna().sum())

Количество пропусков в столбце days_employed: 0


### Шаг 2.2 Проверка данных на аномалии и исправления.

По аналогии со столбцом days_employed в данных могут присутствовать и другие аномарии. Такие артифакты могут возникать из-за некорректного ввода значений при заполнении таблиц (человеческий фактор), при выгрузке данных или из-за ошибок форматирования. Для проверки оставшихся колонок применю к ним метод value_counts(), рассмотрю уникальные значения по каждому столбцу и их количество.

Расчет уникальных значений для колонки children.

In [11]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В колонке children пропущенных значений нет. 

Но есть другие аномалии: 
1. 47 отрицательных значений (-1), что значило бы, что у кого-то -1 ребенок; 
2. 76 значений с количеством детей 20, цифра сильно выделяется из общей выборки и выглядит неправдоподобно.

Таких значений менее 1%, на результаты обработки это сильно не повлияет, но оставлять их некорректно. Заменю значения -1 на 1 и 20 на 2, таким образом присоединю эти группы клиентов к более многочисленным. Для этого использую функцию replace(), убежусь что замена прошла успешно.

In [12]:
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Расчет уникальных значений для колонки dob_years.

In [13]:
df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

В столбце dob_years есть 101 клиент со значением 0 лет. Так как нулевых значений в этом столбце меньше 1% и информация в этой колонке для обработки нам не нужна, то оставлю эти значения без изменений.

Расчет уникальных значений для колонки education.

In [14]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Значения в колонке education необходимо привести к одному регистру. Использую для этого метод str.lower() и проверю, что исправление прошло успешно.

In [15]:
df['education'] = df['education'].str.lower()
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Расчет уникальных значений для колонки education_id.

In [16]:
df['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

В колонке education_id аномалий нет, количество уникальных значений education_id совпадает с количеством уникальных значений education.

Расчет уникальных значений для колонок family_status и family_status_id. Проверять их буду вместе, так как они связаны.

In [17]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [18]:
df['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

Количество уникальных значений в колонках family_status и family_status_id совпадает.
Неявных дубликатов в столбце family_status нет, но я приведу значения к одному регистру, так как это хороший стиль.

In [19]:
df['family_status'] = df['family_status'].str.lower()
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Расчет уникальных значений для колонки gender

In [20]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

В столбце gender есть одно значение отличное от других (XNA). Может означать, что был клиент с неопределенным полом или произошла ошибка при вводе или выводе данных. Пол клиента в анализе участвовать не будет и значение это только одно, оставлю это без изменения.

Расчет уникальных значений для колонки income_type.

In [21]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

В колонке income_type аномалий не вижу.

Расчет уникальных значений для колонки debt.

In [22]:
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

В колонке debt аномалий не вижу.

Расчет уникальных значений для колонки purpose.

In [23]:
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

На данный момент аномалий в колонке purpose не вижу, но возможно из нужно будет сократить в последствии.

### Шаг 2.3. Изменение типов данных.

Для увеличения скорости расчетов и экономии ресурсов компьютера заменю вещественный тип данных в столбце total_income на целочисленный, с помощью метода astype(). Проверю, что все прошло успешно, запросив общую информацию о таблице.

In [24]:
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


Тип данных в столбце total_income изменился на int.

### Шаг 2.4. Удаление дубликатов.

Возможные аномалии в данных были мной исправлены на предидущих этапах исследования.
Проверю есть ли полные дубликаты в таблице, используя функции duplicated() и sum().

In [25]:
print('Количество полных дубликатов:', df.duplicated().sum())

Количество полных дубликатов: 71


В датасете 72 дубликата. Удалю их используя фенкцию drop_duplicates() и метод reset_index(), чтобы не было пропусков в индексах. Проверю результат. 

In [26]:
df = df.drop_duplicates().reset_index(drop=True)
print('Количество явных дубликатов:', df.duplicated().sum())

Количество явных дубликатов: 0


В данном случае считаю корректным удаление только явных дубликатов (функция drop_duplicates()) с учетом исправленных ранее аномалий, по нескольким причинам:
- один клиент мог брать несколько кредитов в одно время под разные цели, тогда его стаж и доход будет одинаковым;
- в таблице нет суммарной характеристики по кредитам на одного конкретного клиента;
- один клиент мог в разное время брать кредит, под одну цель.

Возможные причины появления дубликатов:
- данные об одном кленте могли появиться у разных отделений банков при погашении кредита;
- клиент мог подавать одну заявку на кредит в разных ресурсах (например заявка в приложении, а оформление договора в банке), отсюда данные записанные в разных регистрах;
- ошибки при выгрузке данных из базы.

Ситуацию могла бы проянить дополнительная информация, например, номер кредитного договора или идентификационный номер клиента.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

В рабчем датафрейме присутствуют два набора столбцов, у которых первое значение в текстовом формате показывает смысл, а второе является его идентификационным номером:
- education и education_id;
- family_status и family_status_id.

Хранить оба значения в большом датафрейме не целесообразно, при этом для обработки удобнее будет оставить идентификационные номера, так как при работе с текстовыми значениями:
- усложняется визуальная работа с таблицей;
- увеличивается размер файла и время обработки данных;
- чтобы отфильтровать данные по типу обращения, приходится набирать его полное название, это может привести к ошибкам;
- создание новых категорий и изменение старых отнимает много времени.

Уберу из рабочего датасета избыточную информацию так, чтобы она не потерялась. Для этого создам два новых датафрейма-словоря.

Создание датафрейма-словоря для столбцов education и education_id.

In [27]:
df_education = df[['education','education_id']]                      #создание нового датафрейма из двух колонок рабочего
df_education = df_education.drop_duplicates().reset_index(drop=True) #удаление дубликатов с обновлением индексов
display(df_education.sort_values(by='education_id'))                 #вывод на экран датафрейма отсортированного по id

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


Создание датафрейма-словоря для столбцов family_status и family_status_id.

In [28]:
df_family_status = df[['family_status','family_status_id']]                  #создание нового датафрейма из двух колонок рабочего
df_family_status = df_family_status.drop_duplicates().reset_index(drop=True) #удаление дубликатов с обновлением индексов
display(df_family_status.sort_values(by='family_status_id'))                 #вывод на экран датафрейма отсортированного по id

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Нужная информация сохранена в новых датафреймах df_education и df_family_status.

Теперь можно сократить рабочую таблицу.

Удалю столбцы education и family_status. Выведу на экран первые 5 строк.

In [29]:
df.drop(['education', 'family_status'], axis=1, inplace=True)
display(df.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


Избыточные данные удалены и сохранены в отдельные структуры. Так таблица выглядит компактнее и более удобна для анализа.

### Шаг 2.6. Категоризация дохода.

По результатам исследования мне необходимо ответить на вопрос: "есть ли зависимость между уровнем дохода и возвратом кредита в срок?"
Значения столбца с уровнем дохода клиента принимают разные значения, которые встречаются только один раз. Работать с единичными отрывками и делать из них статистические выводы нельзя. Значит, нужно сгруппировать 
данные, чтобы численности каждой группы хватало для формулировки выводов.

Нужна категоризация. Распределим клиентов по уровню дохода в соответствии с заданием на следующие категории:
- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

Запишу правила классификации клиентов как функцию и проверю ее. 

In [30]:
def total_income_group(total_income_value):
    if 0 <= total_income_value <= 30000:
        return 'E'
    elif 30001 <= total_income_value <= 50000:
        return 'D'
    elif 50001 <= total_income_value <= 200000:
        return 'C'
    elif 200001 <= total_income_value <= 1000000:
        return 'B'
    elif 1000001 <= total_income_value:
        return 'A'
    else:
        return 'Такой категории нет, проверьте входные данные'

print(total_income_group(50))
print(total_income_group(30001))
print(total_income_group(50001))
print(total_income_group(200001))
print(total_income_group(1000001))
print(total_income_group(-1))

E
D
C
B
A
Такой категории нет, проверьте входные данные


Функция total_income_group работает как и задумано.

Создам в рабочем датафрейме столбец total_income_category с категоризацией для столбца total_income. Для этого использую метод  apply(). Выведу на экран первые 5 строк таблицы и уникальные значения нового столбца для проверки результата.

In [31]:
df['total_income_category'] = df['total_income'].apply(total_income_group)
display(df.head())
print('Уникальные значения total_income_category:', df['total_income_category'].unique())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


Уникальные значения total_income_category: ['B' 'C' 'D' 'E' 'A']


Функция total_income_group отработала верно. Категоризация по уровню дохода прошла успешно.

### Шаг 2.7. Категоризация целей кредита.

Аналогично с уровнем дохода требует катекоризации столбец с целью кредита (purpose). Исходя из задания необходимо разбить значения на следубщие категории:
- 'операции с автомобилем';
- 'операции с недвижимостью';
- 'проведение свадьбы';
- 'получение образования'.

Задача осложняется тем, что в столбце purpose данные типа object. Рассмотрю данные еще раз, выведу на экран уникальные значения столбца purpose. Для этого использую функцию value_counts(), количество значений на данный момент мне не нужны, но функция value_counts() выведет значения с новой строки, так удобнее смотреть.

In [32]:
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Для категоризации значений необходимо выделить подстроки в исходной цели кредита, которые можно проверить для каждой из категорий. Использую для этого корни ключевых слов:
- 'операции с автомобилем': 'автом';
- 'операции с недвижимостью': 'недвиж' и 'жил';
- 'проведение свадьбы': 'свад';
- 'получение образования': 'образ'.

Запишу правила классификации клиентов как функцию и проверю ее.

In [33]:
def purpose_group(purpose_value):
    if 'автом' in purpose_value:
        return 'операции с автомобилем'
    elif ('недвиж' in purpose_value) or ('жил' in purpose_value):
        return 'операции с недвижимостью'
    elif 'свад' in purpose_value:
        return 'проведение свадьбы'
    elif 'образ' in purpose_value:
        return 'получение образования'
    else:
        return 'Такой категории нет, проверьте входные данные'

print(purpose_group('заняться образованием'))
print(purpose_group('приобретение автомобиля'))
print(purpose_group('ремонт жилью'))
print(purpose_group('операции со своей недвижимостью '))
print(purpose_group('сыграть свадьбу'))
print(purpose_group('набрать лотерейных билетов'))

получение образования
операции с автомобилем
операции с недвижимостью
операции с недвижимостью
проведение свадьбы
Такой категории нет, проверьте входные данные


Функция purpose_group работает как и задумано.

Создам в рабочем датафрейме столбец purpose_category с категоризацией для столбца purpose. Для этого использую метод apply(). Выведу на экран первые 5 строк таблицы и уникальные значения нового столбца для проверки результата.

In [34]:
df['purpose_category'] = df['purpose'].apply(purpose_group)
display(df.head())
print('Уникальные значения purpose_category:', df['purpose_category'].unique())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


Уникальные значения purpose_category: ['операции с недвижимостью' 'операции с автомобилем'
 'получение образования' 'проведение свадьбы']


Функция purpose_group отработала верно. Категоризация цели кредита прошла успешно.

## Ответы на вопросы.

В ходе исследования необходимо оценить влияние принадлежности клиентов к определенной категории на его платежеспособность. Для такой оценки необходимо выбрать подходящую метрику.

В данном случае отлично подойдет конверсия - отношение целевого показателя к общему. За целевой показатель возьмем количество клиентов не имевших задолженность по кредиту по определенной группе, за целевой общее количество клиентов в группе. Получим конверсию платежеспособности группы клиентов.


### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для группировки по количеству детей использую функцию pivot_table, она создаст наглядный датафрейм с нужной информацией. В качестве входных аргументов буду использовать:
- index='children' - для группировки по индексу;
- columns='debt' - для группировки по столбцам;
- values='total_income'- значения для анализа, можно взять любой столбец, так как мне нужно посчитать количество;
- aggfunc='count' - аггрегирующая функция;
- fill_value=0 - этот параметр в теоретической части мы не проходили, но я использую его, чтобы при отсутствии значений по одной из групп функция выдала мне значение 0, а не NaN.

Сводную таблицу выведу на экран.

In [35]:
children_pivot = df.pivot_table(index=['children'], columns='debt', values='total_income', aggfunc='count', fill_value=0) 
display(children_pivot)

debt,0,1
children,,
0,13028,1063
1,4410,445
2,1926,202
3,303,27
4,37,4
5,9,0


Данные сгруппированы и выведены в удобном формате. 

Теперь необходимо расчитать конверсию. Использую конструкцию try... except... для обработки патенциальных ошибок.

In [36]:
try:
    children_pivot['conversion'] =  children_pivot[0] / (children_pivot[0] + children_pivot[1]) * 100
except:
    print('Ошибка расчета конверсии, проверьте входные значения')
display(children_pivot)

debt,0,1,conversion
children,,,
0,13028,1063,92.456178
1,4410,445,90.834192
2,1926,202,90.507519
3,303,27,91.818182
4,37,4,90.243902
5,9,0,100.000000


#### Вывод 1:

Данные показывают, что:

1. Зависимости между количеством детей и возвратом кредита в срок нет. Группа с минимальной долей должников - клиенты с 5ю детьми (конверсия 100%), группа с максимальной долей должников - клиенты с 4мя детьми (конверсия 90,2%). Но группа с 5ю детьми имеет слишком маленькое число клиентов, в остальных категориях конверсия не превышает 92,5%.

2. Следует отметить, что размер выборки значений с увеличением количеста детей сушественно уменьшается. Т.е. клиенты с детьми реже берут кредиты.

### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для анализа зависимости между семейным положением и возвратом кредита в срок проделаю все те же операции:
- создам сводную таблицу по семейному положению клиентов использую функцию pivot_table - index='family_status_id'. 
- расчитаю конверсию платежеспособности для каждой группы.
- перед выводом на экран "сошью" получившийся датафрейм с созданным ранее датафреймом-словорем df_family_status. Для этого использую функцию merge(). Так результат будет нагляднее. 

In [37]:
family_status_pivot = df.pivot_table(index=['family_status_id'], columns='debt', values='total_income', 
                                     aggfunc='count', fill_value=0)
try:
    family_status_pivot['conversion'] =  family_status_pivot[0] / (family_status_pivot[0] + family_status_pivot[1]) * 100
except:
    print('Ошибка расчета конверсии, проверьте входные значения')

family_status_pivot = family_status_pivot.merge(df_family_status, on='family_status_id', how='right')
display(family_status_pivot)

,family_status_id,0,1,conversion,family_status
0,0,11408,931,92.454818,женат / замужем
1,1,3763,388,90.652855,гражданский брак
2,2,896,63,93.430657,вдовец / вдова
3,3,1110,85,92.887029,в разводе
4,4,2536,274,90.249110,не женат / не замужем


#### Вывод 2:

Данные показывают, что:

1. Зависимости между семейным положением и возвратом кредита в срок нет. Группа с минимальной долей должников - вдовец / вдова (конверсия 93,4%), группа с максимальной долей должников - не женат / не замужем (конверсия 90,2%).

2. Следует отметить, что размер выборки значений для клиентов со статусом "женат/замужем" больше всех остальных на порядок. Это может говорть о том, что криенты состоящие в официальном браке охотнее берут кредит.

### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для анализа зависимости между уровнем дохода и возвратом кредита в срок проделаю следующие операции по аналогии с предыдущими пунктами:

- создам сводную таблицу по категориям уровня дохода клиентов использую функцию pivot_table - index='total_income_category'.
- расчитаю конверсию платежеспособности для каждой группы.
- выведу сводную таблицу на экран.

In [38]:
total_income_pivot = df.pivot_table(index=['total_income_category'], columns='debt', values='total_income', 
                                    aggfunc='count',  fill_value=0)
try:
    total_income_pivot['conversion'] =  total_income_pivot[0] / (total_income_pivot[0] + total_income_pivot[1]) * 100
except:
    print('Ошибка расчета конверсии, проверьте входные значения')
    
display(total_income_pivot)

debt,0,1,conversion
total_income_category,,,
A,23,2,92.000000
B,4685,356,92.937909
C,14656,1360,91.508492
D,329,21,94.000000
E,20,2,90.909091


#### Вывод 3:

Данные показывают, что:

1. Зависимости между уровнем дохода и возвратом кредита в срок нет. Группа с минимальной долей должников - "D" (конверсия 94%), группа с максимальной долей должников - "E" (конверсия 90,9%)..

2. Следует отметить, что размер выборки значений для клиентов с уровнем дохода группы "С" (50001–200000) больше всех остальных на порядок. Это может говорть о том, что криенты со средним уровнем дохода охотнее берут кредит.

### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

Для анализа влияния цели кредита на возврат его в срок проделаю следующие операции по аналогии с предыдущими пунктами:

- создам сводную таблицу по категориям уровня дохода клиентов использую функцию pivot_table - index='purpose_category'.
- расчитаю конверсию платежеспособности для каждой группы.
- выведу сводную таблицу на экран.

In [39]:
purpose_pivot = df.pivot_table(index = ['purpose_category'], columns='debt', values='total_income', 
                               aggfunc='count', fill_value=0)
try:
    purpose_pivot['conversion'] = purpose_pivot[0] / (purpose_pivot[0] + purpose_pivot[1]) * 100
except:
    print('Ошибка расчета конверсии, проверьте входные значения')
display(purpose_pivot)

debt,0,1,conversion
purpose_category,,,
операции с автомобилем,3903,403,90.640966
операции с недвижимостью,10029,782,92.766627
получение образования,3643,370,90.779965
проведение свадьбы,2138,186,91.996558


#### Вывод 4:

Данные показывают, что:

1. Цель кредита не влияет на возврат кредита в срок. Группа с минимальной долей должников - операции с недвижимостью (конверсия 92,8%), группа с максимальной долей должников - операции с автомобилем (конверсия 90,6%).

2. Следует отметить, что размер выборки для операций с недвижимостью больше всех остальных практически в 3 раза. Это может значить, что эта цель наиболее востребована.

## Общий вывод:

В ходе исследования были установлены ответы на четыре вопроса:

1. Зависимости от количества детей и возвратом кредита в срок нет.
2. Зависимости между семейным положением и возвратом кредита в срок нет.
3. Зависимости между уровнем дохода и возвратом кредита в срок нет.
4. Цель кредита не влияет на возврат кредита в срок.

Группа с максимальной долей должников - клиенты с 4мя детьми (конверсия 90,2%). 

Группа с минимальной долей должников - клиенты с доходом категории "D": 30001–50000 (конверсия 94%)

Следует обратить внимание на косвенные результаты исследования. Чаще всего берут кредит:
1. Клиенты без детей.
2. Криенты состоящие в официальном браке.
3. Клиенты с уровнем дохода 50001–200000.
4. Под операции с недвижимостью.

Эти категории могут быть целевыми для банка.

Примечание: Были непонятные моменты в формате названий пунктов. Я их поправила, если это не было нужно делать, верну как было, но мне это не нравилось (мне и сейчас не очень нравится, но в одном из первых уроков просили ничего не менять)